## Important Dependencies with their required versions


In [1]:

# ! pip install SQLAlchemy==2.0.26
# ! pip install langchain==0.1.6
# ! pip install langchain-community==0.0.19
# ! pip install langchain-core==0.1.22
# ! pip install chromadb
# !pip install text_generation

## Imports and Setup and Initializing Agent

In [1]:
# # to auto reload instead of having to restart each time
%load_ext autoreload
%autoreload 2

In [1]:
# from Imports import *
try:
    from Configuration import *
    from Benchmark.TestGenerator import *
    from Benchmark.TestFix import *
    from Benchmark.DecisionMaker import *
    from Benchmark.BugFix import *

    # llm_arb, chat_model_arb = InitializeModelArbiter(os.environ["HUGGINGFACEHUB_API_TOKEN"],repo_id='mistralai/Mixtral-8x7B-Instruct-v0.1')
    # with open(r"Datasets/atcoder_problem_test_cases_description.jsonl", "r") as f:
    #     data = f.read()
    #     jsonObj = json.loads(data)
    jsonObj=pd.read_json(path_or_buf="Datasets/atcoder_problem_test_cases_description.jsonl")
    # jsonObj = HEval_JsonObj
    testGenerator = TestGenerator(GenUnitTestChain, db, jsonObj, globals())
    testGenerator.isHumanEval=False
    # testRegenerator = TestFix(UnitTestFeedbackChain,True,globals())
    # judgeGenerator = DecisionMaker(judgeChain,globals())
    # bugFixGenerator = BugFix(bugFixChain,True,globals())
except Exception as e:
    print(e)
    exit(-1)

C:\Users\DELL\AppData\Roaming\Python\Python311\site-packages\datasets\load.py:2508: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(


## Generate Unit Tests from Code

In [2]:
testGenerator.generate()

File OutputTest/Cases.json created successfully!
File OutputTest/RunningLogs.json created successfully!
Running Test Case  0
FAIL: test_diverse_word_greater_than_26_length (__main__.TestMyFunc.test_diverse_word_greater_than_26_length)
Traceback (most recent call last):
  File "<string>", line 55, in test_diverse_word_greater_than_26_length
AssertionError: None != 'abcdefghijklmnopqrstuvwxyzz'

FAIL: test_diverse_word_less_than_26_length (__main__.TestMyFunc.test_diverse_word_less_than_26_length)
Traceback (most recent call last):
  File "<string>", line 52, in test_diverse_word_less_than_26_length
AssertionError: None != 'abd'

FAIL: test_non_diverse_word (__main__.TestMyFunc.test_non_diverse_word)
Traceback (most recent call last):
  File "<string>", line 58, in test_non_diverse_word
AssertionError: None != 'gotoua'

FAIL: test_non_diverse_word_with_duplicate_characters (__main__.TestMyFunc.test_non_diverse_word_with_duplicate_characters)
Traceback (most recent call last):
  File "<st

Traceback (most recent call last):
  File "C:\Users\DELL\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\DELL\AppData\Local\Temp\ipykernel_19688\1564926404.py", line 1, in <module>
    testGenerator.generate()
  File "f:\College Stuff\GPCollege\LLM-Test-Generator\Benchmark\TestGenerator.py", line 70, in generate
    unittest = self.GenUnitTestChain.invoke(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Python311\Lib\site-packages\langchain\chains\base.py", line 162, in invoke
    raise e
  File "c:\Python311\Lib\site-packages\langchain\chains\base.py", line 156, in invoke
    self._call(inputs, run_manager=run_manager)
  File "c:\Python311\Lib\site-packages\langchain\chains\llm.py", line 103, in _call
    response = self.generate([inputs], run_manager=run_manager)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Python311\Lib

: 

## Judge

In [2]:
judgeGenerator.generate()

File JudgeOutput/JudgmentLogs.json created successfully!
Running Test Case  0
generatedJudgement:  <s> [INST] Given the Python code below, its description, and an error-producing test case with the associated error message, determine the cause of the error. Use the information provided to decide whether the error is due to a bug in the Python code itself or in the test case. Provide a reasoned conclusion.

- Code Under Test:

from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    for idx, elem in enumerate(numbers):
        for idx2, elem2 in enumerate(numbers):
            if idx != idx2:
                distance = abs(elem - elem2)
                if distance < threshold:
                    return True

    return False

- Description of Functionality:
    Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elem

In [ ]:
judgeOut=1
if judgeOut==1:
    pass
else:
    testRegenerator.generate()

In [13]:
txt="""S=input()\nX=input()\nl='abcdefghijklmnopqrstuvwxyz'\nP=[]\nif S==l[::-1]:\n    print(-1)\nelse:\n    ans=''\n    if len(S)<26:\n        for i in range(26):\n            if l[i] not in S:\n                ans=S+l[i]\n                print(ans)\n                break\n    else:\n        m=S[::-1]\n        x=m[0]\n        for i in range(26):\n            if x>m[i]:\n                u=m[i]\n                for j in S:\n                    if j==u:\n                        break\n                    else:\n                        ans+=j\n                for j in l:\n                    if j not in ans and j!=u:\n                        ans+=j\n                        print(ans)\n                        exit()\n            else:\n                x=m[i]"""

import re
def replace_input(code):
    # find all the lines that have input() in them
    lines = code.split("\n")
    variableNames = []
    # find the variable name on the left hand side of the assignment
    variablematches = re.finditer(r"(.+?)\s*=\s*input\(\)\s*", code)
    for matcher in variablematches:
        variableNames.append(matcher.group(1).strip())
    functionheader = "def(" + ",".join(variableNames) + "):"
    # replace any line containing input with ""
    for i in range(len(lines)):
        if "input()" in lines[i]:
            lines[i] = ""
        lines[i] = "\t" + lines[i]

    # add function header and format code
    code = functionheader + "\n" + "\n".join(lines)
    return code

bug=replace_input(txt)
print(bug)

def(S):
	
	l='abcdefghijklmnopqrstuvwxyz'
	P=[]
	if S==l[::-1]:
	    print(-1)
	else:
	    ans=''
	    if len(S)<26:
	        for i in range(26):
	            if l[i] not in S:
	                ans=S+l[i]
	                print(ans)
	                break
	    else:
	        m=S[::-1]
	        x=m[0]
	        for i in range(26):
	            if x>m[i]:
	                u=m[i]
	                for j in S:
	                    if j==u:
	                        break
	                    else:
	                        ans+=j
	                for j in l:
	                    if j not in ans and j!=u:
	                        ans+=j
	                        print(ans)
	                        exit()
	            else:
	                x=m[i]
